# Auto Loan Prediction Database - Delicia Jathanna

## CSCI 3287 

In [2]:
import sqlite3
import zipfile
import pandas as pd

## Connecting to my database & Creating the tables

In [3]:
df_auto = pd.read_csv('auto_dataset.zip')

In [4]:
conn = sqlite3.connect("autoloansdb.db")
cur = conn.cursor()

In [5]:
drop_table_jobs = conn.execute('DROP TABLE IF EXISTS jobs;')

In [6]:
create_jobs_table = conn.execute('''
CREATE TABLE IF NOT EXISTS jobs
	(
	  occupation VARCHAR(50) NOT NULL, --Occupation of the application
      salary INT NOT NULL, --Monthly income of the applicant
      credit_score INTEGER NOT NULL --Credit Score of the applicant
	);
''')

In [7]:
drop_table_loan = conn.execute('DROP TABLE IF EXISTS loan;')

In [8]:
create_loan_table = conn.execute('''
CREATE TABLE IF NOT EXISTS loan
	(
	  loan_id VARCHAR(50) PRIMARY KEY, --ID of the loan (primary key)
      occupation VARCHAR(50), --Occupation of the applicant (foreign key)
      FOREIGN KEY (occupation) REFERENCES jobs (occupation)
	);
''')

In [9]:
drop_table_loan_status = conn.execute('DROP TABLE IF EXISTS loan_status;')

In [10]:
create_loan_status_table = conn.execute('''
CREATE TABLE IF NOT EXISTS loan_status
	(
	  loan_id VARCHAR(50), --ID of the loan (foreign key)
      loan_status VARCHAR(50) CHECK (loan_status = 'Y' OR loan_status = 'N'), --Status of the loan
      credit_score INTEGER, --Credit Score of the applicant (foreign key)
      FOREIGN KEY (loan_id) REFERENCES loan (loan_id)
      FOREIGN KEY (credit_score) REFERENCES jobs (credit_score)
	);
''')

In [11]:
drop_table_loan_term = conn.execute('DROP TABLE IF EXISTS loan_term;')

In [12]:
create_loan_term_table = conn.execute('''
CREATE TABLE IF NOT EXISTS loan_term
	(
	  loan_id VARCHAR(50), --ID of the loan (foreign key)
      loan_term INTEGER, --Term of the loan in months 
      FOREIGN KEY (loan_id) REFERENCES loan_status (loan_id) ON INSERT CASCADE
	);
''')

In [13]:
### Filling the tables with the data from the dataset

In [14]:
for row in df_auto.to_dict('records'):
    cmd = ('''INSERT INTO jobs(occupation, salary, credit_score)\
    VALUES ( :Occupation, :Monthly_Income, :Credit_Score) ''' )
    conn.execute(cmd, row)

In [15]:
for row in df_auto.to_dict('records'):
    cmd = ('''INSERT INTO loan(loan_id, occupation)\
    VALUES ( :Loan_ID, :Occupation) ''' )
    conn.execute(cmd, row)

In [16]:
for row in df_auto.to_dict('records'):
    cmd = ('''INSERT INTO loan_status(loan_id, loan_status, credit_score)\
    VALUES ( :Loan_ID, :Loan_Status, :Credit_Score) ''' )
    conn.execute(cmd, row)

In [17]:
for row in df_auto.to_dict('records'):
    cmd = ('''INSERT INTO loan_term(loan_id, loan_term)\
    VALUES ( :Loan_ID, :Loan_Amount_Term) ''' )
    conn.execute(cmd, row)

### Display the initial data in the tables

In [18]:
res = cur.execute("SELECT * FROM jobs LIMIT 10")
res.fetchall()

[('Teacher', 4000, 750),
 ('Engineer', 6000, 800),
 ('Nurse', 25000, 650),
 ('Doctor', 8000, 900),
 ('Businessman', 10000, 700),
 ('Engineer', 5000, 750),
 ('Teacher', 3500, 650),
 ('Nurse', 2000, 600),
 ('Businessman', 12000, 850),
 ('Doctor', 9000, 800)]

In [19]:
res = cur.execute("SELECT * FROM loan LIMIT 10")
res.fetchall()

[('LP001002', 'Teacher'),
 ('LP001003', 'Engineer'),
 ('LP001005', 'Nurse'),
 ('LP001006', 'Doctor'),
 ('LP001008', 'Businessman'),
 ('LP001011', 'Engineer'),
 ('LP001013', 'Teacher'),
 ('LP001014', 'Nurse'),
 ('LP001018', 'Businessman'),
 ('LP001020', 'Doctor')]

In [20]:
res = cur.execute("SELECT * FROM loan_status LIMIT 10")
res.fetchall()

[('LP001002', 'Y', 750),
 ('LP001003', 'Y', 800),
 ('LP001005', 'Y', 650),
 ('LP001006', 'Y', 900),
 ('LP001008', 'Y', 700),
 ('LP001011', 'Y', 750),
 ('LP001013', 'N', 650),
 ('LP001014', 'N', 600),
 ('LP001018', 'Y', 850),
 ('LP001020', 'Y', 800)]

In [21]:
res = cur.execute("SELECT * FROM loan_term LIMIT 10")
res.fetchall()

[('LP001002', 36),
 ('LP001003', 36),
 ('LP001005', 36),
 ('LP001006', 36),
 ('LP001008', 36),
 ('LP001011', 36),
 ('LP001013', 0),
 ('LP001014', 0),
 ('LP001018', 36),
 ('LP001020', 18)]

# Index on loan_id

In [22]:
drop_loan_id_index = conn.execute("DROP INDEX IF EXISTS loan_id_index")

In [23]:
loan_id_index = conn.execute("CREATE INDEX loan_id_index ON loan(loan_id)")

# Queries

Now that we've added some indexes and triggers to ensure the dataset runs smoothly and meets the requirements, lets run some queries on them.

This first query will join all the ables together. This will take a subquery or using CTEs because not all of the tables have the same relationships. 

In [24]:
all_table_join = cur.execute("""
    WITH 
    CTE1 AS (
            SELECT loan_id AS Loan_ID, jobs.occupation AS occupation, jobs.salary AS Salary, jobs.credit_score AS [Credit Score]
            FROM loan 
                JOIN jobs on jobs.occupation = loan.occupation
            ),
    CTE2 AS (
            SELECT loan.loan_id, loan_status AS Status
            FROM loan_status
                JOIN loan on loan.loan_id = loan_status.loan_id
            ),
    CTE3 AS (
            SELECT loan.loan_id, loan_term AS Term
            FROM loan_term 
                JOIN loan on loan.loan_id = loan_term.loan_id
            ) 
    SELECT CTE1.loan_id, Occupation, Salary, [Credit Score], CTE2.Status, CTE3.Term
    FROM  CTE1
          JOIN CTE2 ON CTE1.loan_id = CTE2.loan_id
          JOIN CTE3 ON CTE1.loan_id = CTE3.loan_id
    GROUP BY Occupation
    ORDER BY Occupation
    LIMIT 10;
    """)

all_table_join.fetchall()

[('LP001770', 'Account Executive', 6000, 720, 'Y', 24),
 ('LP001146', 'Account Manager', 5800, 720, 'Y', 36),
 ('LP001034', 'Accountant', 4500, 690, 'Y', 24),
 ('LP001043', 'Architect', 5500, 740, 'Y', 36),
 ('LP001784', 'Attorney', 1200, 800, 'Y', 36),
 ('LP001532', 'Automotive Mechanic', 3000, 640, 'N', 0),
 ('LP001579', 'Bank Teller', 2700, 610, 'N', 0),
 ('LP001405', 'Barista', 2000, 610, 'N', 0),
 ('LP001492', 'Bartender', 2500, 610, 'N', 0),
 ('LP001610', 'Business Analyst', 6500, 750, 'Y', 36)]

##### Now lets run some requries that show what salary, credit score, and occupation are more favorable to be approved for an auto loan.

In [25]:
favorable_options = cur.execute("""
    WITH 
    CTE1 AS (
            SELECT loan_id AS Loan_ID, jobs.occupation AS occupation, jobs.salary AS Salary, jobs.credit_score AS [Credit Score]
            FROM loan 
                JOIN jobs on jobs.occupation = loan.occupation
            ),
    CTE2 AS (
            SELECT loan.loan_id, loan_status AS Status
            FROM loan_status
                JOIN loan on loan.loan_id = loan_status.loan_id
            )
    SELECT CTE1.loan_id, Occupation, Salary, [Credit Score], CTE2.Status 
    FROM  CTE1
        JOIN CTE2 on CTE1.loan_id = CTE2.loan_id
    WHERE Status = 'Y'
    GROUP BY Occupation
    ORDER BY [Credit Score] desc
    LIMIT 10;
    """)

favorable_options.fetchall()

[('LP001006', 'Doctor', 8000, 900, 'Y'),
 ('LP001066', 'Pilot', 10000, 820, 'Y'),
 ('LP001325', 'Physician', 15000, 820, 'Y'),
 ('LP001379', 'Optometrist', 9000, 820, 'Y'),
 ('LP001030', 'Lawyer', 9000, 820, 'Y'),
 ('LP001280', 'Pharmacist', 10000, 810, 'Y'),
 ('LP001493', 'Physician Assistant', 7500, 800, 'Y'),
 ('LP001543', 'Financial Manager', 9000, 800, 'Y'),
 ('LP001029', 'Entrepreneur', 8000, 800, 'Y'),
 ('LP001404', 'Business Owner', 15000, 800, 'Y')]

##### Now lets do the opposite

In [26]:
unfavorable_options = cur.execute("""
    WITH 
    CTE1 AS (
            SELECT loan_id AS Loan_ID, jobs.occupation AS occupation, jobs.salary AS Salary, jobs.credit_score AS [Credit Score]
            FROM loan 
                JOIN jobs on jobs.occupation = loan.occupation
            ),
    CTE2 AS (
            SELECT loan.loan_id, loan_status AS Status
            FROM loan_status
                JOIN loan on loan.loan_id = loan_status.loan_id
            )
    SELECT CTE1.loan_id, Occupation, Salary, [Credit Score], CTE2.Status AS Status
    FROM  CTE1
    JOIN CTE2 on CTE1.loan_id = CTE2.loan_id
    WHERE Status = 'N'
    GROUP BY Occupation
    ORDER BY [Credit Score] 
    LIMIT 10;
    """)

unfavorable_options.fetchall()

[('LP001073', 'Writer', 2000, 500, 'N'),
 ('LP001041', 'Electrician', 2500, 550, 'N'),
 ('LP001086', 'Musician', 2500, 550, 'N'),
 ('LP001439', 'Waiter/Waitress', 1500, 560, 'N'),
 ('LP001391', 'Dental Assistant', 2500, 590, 'N'),
 ('LP001516', 'Retail Salesperson', 2000, 590, 'N'),
 ('LP001131', 'Construction Worker', 2500, 600, 'N'),
 ('LP001195', 'Customer Service', 2500, 600, 'N'),
 ('LP001465', 'Hair Stylist', 2000, 600, 'N'),
 ('LP001109', 'Mechanic', 2500, 600, 'N')]

# Triggers and Indexes
We will now Add some triggers and indexes to the data so that the queries run smoother and we can ensure that when a foreign key gets deleted, it gets deleted in every table.

In [27]:
drop_fk_occupation_trigger = conn.execute("DROP TRIGGER IF EXISTS fk_occupation_delete")

In [28]:
fk_occupation_trigger = conn.execute('''
    CREATE TRIGGER fk_occupation_delete
    AFTER DELETE ON jobs 
    FOR EACH ROW
    BEGIN
        DELETE FROM loan
        WHERE occupation = old.occupation;
    END;
    ''')

In [29]:
drop_fk_loanid_trigger = conn.execute("DROP TRIGGER IF EXISTS fk_loanid_delete")

In [30]:
fk_loanid_trigger = conn.execute('''
    CREATE TRIGGER fk_loanid_delete
    AFTER DELETE ON loan 
    FOR EACH ROW
    BEGIN
        DELETE FROM loan_term
        WHERE loan_id = old.loan_id;
        DELETE FROM loan_status
        WHERE loan_id = old.loan_id;
    END;
    ''')

In [31]:
drop_fk_score_trigger = conn.execute("DROP TRIGGER IF EXISTS fk_score_delete")

In [32]:
fk_score_trigger = conn.execute('''
    CREATE TRIGGER fk_score_delete
    AFTER DELETE ON jobs 
    FOR EACH ROW
    BEGIN
        DELETE FROM loan_status
        WHERE credit_score = old.credit_score;
    END;
    ''')

### Testing the occupation foreign key trigger

In [33]:
res = cur.execute("DELETE FROM jobs WHERE occupation = 'Teacher'")

In [34]:
res = cur.execute("SELECT * FROM jobs LIMIT 10")
res.fetchall()

[('Engineer', 6000, 800),
 ('Nurse', 25000, 650),
 ('Doctor', 8000, 900),
 ('Businessman', 10000, 700),
 ('Engineer', 5000, 750),
 ('Nurse', 2000, 600),
 ('Businessman', 12000, 850),
 ('Doctor', 9000, 800),
 ('Engineer', 6000, 700),
 ('Salesperson', 3000, 600)]

In [35]:
res = cur.execute("SELECT * FROM loan LIMIT 10")
res.fetchall()

[('LP001003', 'Engineer'),
 ('LP001005', 'Nurse'),
 ('LP001006', 'Doctor'),
 ('LP001008', 'Businessman'),
 ('LP001011', 'Engineer'),
 ('LP001014', 'Nurse'),
 ('LP001018', 'Businessman'),
 ('LP001020', 'Doctor'),
 ('LP001024', 'Engineer'),
 ('LP001027', 'Salesperson')]

### Testing the loan_id trigger

In [36]:
res = cur.execute("DELETE FROM loan WHERE loan_id = 'LP001005'")

In [37]:
res = cur.execute("SELECT * FROM loan LIMIT 10")
res.fetchall()

[('LP001003', 'Engineer'),
 ('LP001006', 'Doctor'),
 ('LP001008', 'Businessman'),
 ('LP001011', 'Engineer'),
 ('LP001014', 'Nurse'),
 ('LP001018', 'Businessman'),
 ('LP001020', 'Doctor'),
 ('LP001024', 'Engineer'),
 ('LP001027', 'Salesperson'),
 ('LP001029', 'Entrepreneur')]

In [38]:
res = cur.execute("SELECT * FROM loan_term LIMIT 10")
res.fetchall()

[('LP001003', 36),
 ('LP001006', 36),
 ('LP001008', 36),
 ('LP001011', 36),
 ('LP001014', 0),
 ('LP001018', 36),
 ('LP001020', 18),
 ('LP001024', 36),
 ('LP001027', 0),
 ('LP001029', 36)]

In [39]:
res = cur.execute("SELECT * FROM loan_status LIMIT 10")
res.fetchall()

[('LP001003', 'Y', 800),
 ('LP001006', 'Y', 900),
 ('LP001008', 'Y', 700),
 ('LP001014', 'N', 600),
 ('LP001018', 'Y', 850),
 ('LP001020', 'Y', 800),
 ('LP001024', 'Y', 700),
 ('LP001027', 'N', 600),
 ('LP001029', 'Y', 800),
 ('LP001030', 'Y', 820)]

### Testing the credit_score trigger

In [40]:
res = cur.execute("DELETE FROM jobs WHERE credit_score <= 700 ")

In [41]:
res = cur.execute("SELECT * FROM jobs LIMIT 10")
res.fetchall()

[('Engineer', 6000, 800),
 ('Doctor', 8000, 900),
 ('Engineer', 5000, 750),
 ('Businessman', 12000, 850),
 ('Doctor', 9000, 800),
 ('Entrepreneur', 8000, 800),
 ('Lawyer', 9000, 820),
 ('Doctor', 12000, 900),
 ('Accountant', 5000, 720),
 ('Software Dev', 7000, 780)]

In [42]:
res = cur.execute("SELECT * FROM loan LIMIT 10")
res.fetchall()

[('LP001006', 'Doctor'),
 ('LP001020', 'Doctor'),
 ('LP001029', 'Entrepreneur'),
 ('LP001030', 'Lawyer'),
 ('LP001032', 'Doctor'),
 ('LP001036', 'Software Dev'),
 ('LP001043', 'Architect'),
 ('LP001052', 'Real Estate Ag'),
 ('LP001066', 'Pilot'),
 ('LP001091', 'Software Developer')]

In [43]:
res = cur.execute("SELECT * FROM loan_status LIMIT 10")
res.fetchall()

[('LP001006', 'Y', 900),
 ('LP001020', 'Y', 800),
 ('LP001029', 'Y', 800),
 ('LP001030', 'Y', 820),
 ('LP001032', 'Y', 900),
 ('LP001036', 'Y', 780),
 ('LP001043', 'Y', 740),
 ('LP001052', 'Y', 760),
 ('LP001066', 'Y', 820),
 ('LP001091', 'Y', 800)]

Now that we've ensured that when a foreign key gets deleted, it deletes in every table, lets add a trigger to the loan status table that a loan cannot be approved if the credit score is below 675. 

In [44]:
drop_loan_status_trigger = conn.execute("DROP TRIGGER IF EXISTS loan_status_trigger")

In [45]:
loan_status_trigger = conn.execute('''
    CREATE TRIGGER loan_status_trigger
    AFTER INSERT ON loan_status 
    FOR EACH ROW
    BEGIN
        SELECT CASE 
            WHEN ((SELECT loan_status FROM loan_status WHERE loan_status.loan_id = NEW.loan_id AND loan_status.credit_score < 675) = 'Y') 
            THEN RAISE(ABORT, 'Credit Score must be above 675 to be approved for a loan') 
        END;
    END;
    ''')

In [46]:
test_loan_status_trigger = conn.execute("INSERT INTO loan_status(loan_id, credit_score, loan_status) VALUES('LP002000', 650, 'Y')")

IntegrityError: Credit Score must be above 675 to be approved for a loan

In [47]:
cur.close()